The stop frequency model assigns to each tour the number of intermediate destinations a person will travel to on each leg of the tour from the origin to tour primary destination and back. Intermediate stops are not modeled for drive-transit tours

In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
import validation_data_input
import psrc_theme


import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data
land_use = pd.read_csv(config['p_survey_landuse'])

per_data = validation_data_input.get_persons_data(['ptype'])
hh_data = validation_data_input.get_households_data(['auto_ownership','home_zone_id'])
tour_data = validation_data_input.get_tours_data()

In [3]:
tour_data['stop_frequency_out'] = tour_data['stop_frequency'].apply(lambda x: x[0])
tour_data['stop_frequency_in'] = tour_data['stop_frequency'].apply(lambda x: x[5])

# add auto_ownership with 4+
hh_data['auto_ownership_simple'] = hh_data['auto_ownership'].apply(lambda x: "4+" if x>=4.0 else str(x))
# household density groups
var_group = land_use['log_hh_1'].quantile([.00, .125, .25, .50, .75,1.00])
land_use['household_density_bin'] = pd.cut(land_use['log_hh_1'], bins=var_group.tolist(),
                                           labels=['very low', 'low', 'medium', 'medium-high', 'high'])

# add person type labels
ptype_cat = {1: "1: Full-Time Worker",
             2: "2: Part-Time Worker",
             3: "3: University Student",
             4: "4: Non-Working Adult Age <65",
             5: "5: Non-Working Adult Age 65+",
             6: "6: High School Student Age 16+",
             7: "7: Child Age 5-15",
             8: "8: Child Age 0-4"}
per_data['ptype_label'] = per_data['ptype'].map(ptype_cat)

hh_data = hh_data.merge(land_use[['log_hh_1','household_density_bin','zone_id']],how="left",left_on='home_zone_id',right_on='zone_id')
per_data = per_data.merge(hh_data[['household_id','auto_ownership','auto_ownership_simple','log_hh_1','household_density_bin','source']],
                          how='left', on=['household_id','source']) # get auto ownership from hh data

tour_data = tour_data.merge(per_data, how='left', on=['person_id','household_id','source'])

In [4]:
#| layout-ncol: 2
df_plot = tour_data.groupby(['source','stop_frequency_out'])['trip_weight_2017_2019'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="stop_frequency_out", y="percentage", color="source",barmode="group",
             title="outbound stop frequency")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=400, yaxis=dict(tickformat=".1%"))
fig.show()

df_plot = tour_data.groupby(['source','stop_frequency_in'])['trip_weight_2017_2019'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="stop_frequency_in", y="percentage", color="source",barmode="group",
             title="inbound stop frequency")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=400, yaxis=dict(tickformat=".1%"))
fig.show()

## stop frequency by tour type

In [5]:
def plot_stop_frequency(df: pd.DataFrame, in_out: str, grp_var: str, n_nol: int, height: int, title_name: str):
    df_plot = df.groupby(['source',grp_var,in_out])['trip_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',grp_var], group_keys=False)['trip_weight_2017_2019']. \
        apply(lambda x: x / float(x.sum()))

    fig = px.bar(df_plot[df_plot[in_out]!="0"],
                 x=in_out, y="percentage", color="source",barmode="group",
                 facet_col=grp_var, facet_col_wrap=n_nol,
                 title=title_name)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
    fig.update_layout(height=height, width=700)
    fig.show()

In [6]:
plot_stop_frequency(tour_data,'stop_frequency_out','tour_type',3,900,"outbound stop frequency by tour type")
plot_stop_frequency(tour_data,'stop_frequency_in','tour_type',3,900,"inbound stop frequency by tour type")